## Interactive map with custom gtif

#### Initial imports

In [ ]:
#imports
import os
import json
from collections import OrderedDict
# import folium
# from folium import plugins
import rioxarray as rxr
import xarray as xr
import earthpy as et
import earthpy.spatial as es
from pathlib import Path
# from ipyleaflet import *
# from ipyleaflet import Map, basemaps, basemap_to_tiles
import leafmap.leafmap as leafmap
import rasterio as rio
from ipyvuetify.extra import FileInput
import ipywidgets as widgets
from ipywidgets import Button, Layout, SelectMultiple, VBox, Label, Checkbox, HTML, Box, Dropdown, Select
from ipyfilechooser import FileChooser
# from IPython.display import display, HTML
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output
from shapely import orient_polygons, Polygon

#### Read example data files

In [ ]:
#fetch example data
# datapath = r'\LocalPACESearch\target_data_tif\level2_bgc\lake_erie_west\2025-12-10\15_05_53_PACE_OCI_L2_BGC\PACE_OCI.20250604T185022.L2.OC_BGC.V3_1.tif'

In [ ]:
#filechooser file selection

#home path 
home = r'I:'

fc = FileChooser(f'{home}/')

fc.sandbox_path = f'{home}/'

# fc.filter_pattern = ['*.nc']

fc.show_only_dirs = True #only allow the user to select a target directory for now, they can multi-select filenames next

fc.title = '<h3>Select Directory</h3>'
fc.width='100%'

display(fc)


def update_fname():
    global fname
    fname = fc.selected
    return

fc.register_callback(update_fname)

In [ ]:
fname #this is the chosen folder of files

In [ ]:
# file browser
# go through the selected folder, displaying all valid .tif files
# filenames = [] #list of filenames

all_paths = []

for root, dirs, files in os.walk(fname):
    for file in files:
        if not file in all_paths and ".tif" in file:
            all_paths.append(os.path.join(root, file))
if not all_paths:
    print("No files found.")
else:
    #sort the paths in alphabetical order
    all_paths.sort()
    # print(all_paths)

    #title label
    title_label = HTML(value="<h3>Select Files to Visualize</h3>")

    #select all checkbox
    select_all_check = Checkbox(
        value = False,
        description="Select all",
        margin="20px",
        align_self = 'center',
        indent=False,
        padding='4px 0px',
        disabled=False
    )

    # display(title_layout)
    select_label = Label("List of available files:", margin='20px')

    #multiselect files to visualize
    sel_layout = Layout(width='auto', height='100px', margin='20px 2px')

    sel = SelectMultiple(
        options=all_paths,
        layout = sel_layout,
        disabled=False
    )

    def get_selected(*args):

        global selected_files
        selected_files = sel.value

        #set up the pandas dataframe with formatting
        file_pdframe = pd.DataFrame(selected_files, columns=['Selected Files']).reset_index().rename(columns={'index':'Index'})
        # file_pdframe.index.name = 'Index'
        pd.set_option('display.max_colwidth', None)

        file_pdframe_styled = file_pdframe.style.set_table_styles([
            dict(selector='th', props=[('text-align', 'center'),]),
            dict(selector='td', props=[('text-align', 'center'),])
        ]).hide()

        # file_pdframe.style.set_properties(**{'text-align': 'center'})

        display(file_pdframe_styled)

        # display()
        # clear_output(wait=True)

    #confirm buttom
    confirm_button = Button(description="Confirm Selection", margin="20px")
    confirm_button.on_click(get_selected)

    box_layout = Layout(
        display='flex',
        flex_flow='column',
        align_items='flex-start',
        width='80%',
        margin='10px',
        # padding_bottom='50px'
    )

    display(Box([title_label, select_label, sel, confirm_button], layout = box_layout))

    # sel.label = "List of available files"
    # display(sel)

# fi = FileInput()

# def on_file_upload(change):
#     file = fi.get_files()[0]
#     print(file)

# fi.observe(on_file_upload, names="file_info")


In [ ]:
selected_files # the list of selected files to double-check

In [ ]:
#now we need to plot all selected files!

#### Processing the polygon file

In [ ]:
#process the polygon file
polygonCoords = [] #to store the polygon coordinates

In [ ]:
#filechooser file selection for the polygon

fpoly = FileChooser(f'{home}/')

fpoly.sandbox_path = f'{home}/'

fpoly.filter_pattern = ['*.geojson']

fpoly.title = '<h3>Select GeoJson File</h3>'
fpoly.width='100%'

display(fpoly)

def update_fpolyname():
    global fpolyname
    fpolyname = fpoly.selected
    return

fpoly.register_callback(update_fpolyname)

In [ ]:
fpolyname #the polygon filename

##### Polygon point processing (not necessary with leafmap, can just use `leafmap.add_geojson`)
- purpose is just to get the centroid of the area of interest to center the map

In [ ]:
#functions to help process coordinates & polygon file
def CCWPolyOrient (polyPoints):
    ccw_polypts = Polygon(polyPoints) #convert the list of points to a shapely polygon obj
    ccw_polypts = orient_polygons(ccw_polypts) #use shapely's builtin orientpolygons to ensure exterior points are all ccw, following convention

    # print(ccw_polypts)

    #refill the points list with the reoriented point order
    ccw_polypts = [(pt[0], pt[1]) for pt in ccw_polypts.exterior.coords]

    # print(ccw_polypts)
    return ccw_polypts

def processPolygonFile(polyJsonFile):
    #read shape file
    try:
        shapef_name = open(polyJsonFile)
        shapef = json.load(shapef_name)
        shapef = shapef["features"]
        shapef_geo = shapef[0]["geometry"] #extract geometry entry

        all_coordPoints = [] #make sure the all_coordPoints array is empty

        #IMPORTANT: Notes on the required format for coordinate points file
            #coordpoints must be list in CCW order
            #first coordpoint must be the same as the last point (to form a closed polygon)

        #read tuple coordinate points into the array
        for i in shapef_geo["coordinates"][0][0]:
            x, y = float(i[0]), float(i[1])
            all_coordPoints.append((y, x)) #assumes the file is CCW by default, and appends points to the end of the list
        
        #ensure the points are in CCW order
        all_coordPoints = CCWPolyOrient(all_coordPoints)
        return all_coordPoints
    except Exception as e:
        return []

In [ ]:
polygonCoords = processPolygonFile(fpolyname)
polygonCoords #check that the polygon was processed correctly

In [ ]:
polygon = Polygon(polygonCoords) #make into polygon object

#### Start displaying the map

##### Displaying polygon area

In [ ]:
#check the polygon centroid (to center the map)
cx, cy = polygon.centroid.x, polygon.centroid.y
(cx, cy)

In [ ]:
#empty map
map = leafmap.Map(toolbar_control=False, layer_control=False, draw_control=False, attribution_control=False, widget_control=False)
# map.add_basemap("OpenStreetMap")
map.set_center(cy, cx)
map.zoom = 8

#if deciding to use ipyleaflet instead, use these as params
# center = [cx, cy],
# zoom = 8

# map #show empty default map

In [ ]:
#plot polygon
polylayername = fpolyname.split('\\')[-1].split('.geojson')[-2]
polylayername #get the layer name for the polygon to be displayed on the map

In [ ]:
#add to map
map.add_geojson(fpolyname, layer_name=polylayername, style={'color': 'white', 'weight': 3, 'fill': True, 'fillColor': 'white', 'fillOpacity': 0.1}, zoom_to_layer=False, info_mode=None,)
# map

##### Displaying map layers

In [ ]:
#test with a random datafile first
# datafile = selected_files[0]
# layer_name = datafile.split('\\')[-1].split('.tif')[-2]
# map.add_raster(datafile, cmap="jet", layer_name = layer_name, opacity=1, zoom_to_layer = False)

In [ ]:
#iterate through all files and plot
layer_colourmaps = {}

for datafile in selected_files:
    layer_name = datafile.split('\\')[-1].split('.tif')[-2]
    map.add_raster(datafile, cmap="jet", layer_name = layer_name, opacity=1, zoom_to_layer = False, toolbar_control=False, widget_control=False)
    # rasters.append([datafile, "jet", layer_name, 1, False, False])

    #save colourmap of each data file (to add legend later)
    #save the layer min, max, and file name
    layer_min, layer_max = leafmap.image_min_max(datafile)
    layer_colourmaps[datafile] = (layer_min, layer_max)
    # map.add_colormap(cmap="jet", vmin=layer_min, vmax=layer_max, label=f"{layer_name}\nChlor_a conc. (mg per cubic m)", width=5, height=0.4, orientation='horizontal', )

In [ ]:
layer_colourmaps

In [ ]:
#add inspection gui to check data values
map.add_inspector_gui(opened=True, position='bottomleft')

#add isolated layer control widget to toggle visibility of layers
#NO SCROLL BAR! I recommend not opening more than 13 datafiles at a time for viewing
map.add_layer_manager(position='topright', opened = True)

#add scale to the bottom right
# map.scale_control(position='bottomright')

In [ ]:
map.controls #show all the controls that were added to the map so far

In [ ]:
map #display map

##### Displaying a single colourbar and isolated data

In [ ]:
#sort alphabetical order (will be in sequence from oldest to newest!)
layer_colourmaps = OrderedDict(sorted(layer_colourmaps.items())) 
layer_colourmaps

In [ ]:
#select a file to get the colourbar for display

#title label
title_label = HTML(value="<h3>Select Data to Show Colourbar</h3>")

#select all checkbox
# select_all_check = Checkbox(
#     value = False,
#     description="Select all",
#     margin="20px",
#     align_self = 'center',
#     indent=False,
#     padding='4px 0px',
#     disabled=False
# )

# display(title_layout)
select_label = Label("List of available data:", margin='20px')

#multiselect files to visualize
sel_layout = Layout(width='auto', height='100px', margin='20px 2px')

#map the file paths to filenames
colourmap_filenames = {}
for layername in list(layer_colourmaps.keys()):
    colourmap_filenames[layername.split('\\')[-1].split('.tif')[-2]] = layername

#create selection widget
sel = Select(
    options=list(colourmap_filenames.keys()),
    layout = sel_layout,
    disabled=False
)

def get_selected(*args):

    global selected_file
    selected_file = sel.value

    #set up the pandas dataframe with formatting
    # file_pdframe = pd.DataFrame(selected_files, columns=['Selected Data']).reset_index().rename(columns={'index':'Index'})
    # # file_pdframe.index.name = 'Index'
    # pd.set_option('display.max_colwidth', None)

    # file_pdframe_styled = file_pdframe.style.set_table_styles([
    #     dict(selector='th', props=[('text-align', 'center'),]),
    #     dict(selector='td', props=[('text-align', 'center'),])
    # ]).hide()

    # file_pdframe.style.set_properties(**{'text-align': 'center'})

    display(selected_file)

    # display()
    # clear_output(wait=True)

#confirm buttom
confirm_button = Button(description="Confirm Selection", margin="20px")
confirm_button.on_click(get_selected)

box_layout = Layout(
    display='flex',
    flex_flow='column',
    align_items='flex-start',
    width='80%',
    margin='10px',
    # padding_bottom='50px'
)

display(Box([title_label, select_label, sel, confirm_button], layout = box_layout)) 

In [ ]:
colourmap_filenames

In [ ]:
selected_path = colourmap_filenames[selected_file]
selected_path

In [ ]:
#fetch the corresponding min/max data from before and create colourbar/map
min_cmap, max_cmap =layer_colourmaps[selected_path][0], layer_colourmaps[selected_path][1]
min_cmap, max_cmap

In [ ]:
# from leafmap import leafmap
#new empty map
map_single = leafmap.Map(toolbar_control=False, layer_control=False, draw_control=False, attribution_control=False, widget_control=False)
map_single.add_basemap("OpenStreetMap")

map_single.set_center(cy, cx)
map_single.zoom = 8

In [ ]:
#add back the polygon layer
map_single.add_geojson(fpolyname, layer_name=polylayername, style={'color': 'white', 'weight': 3, 'fill': True, 'fillColor': 'white', 'fillOpacity': 0.1}, zoom_to_layer=False, info_mode=None,)

#add the single data layer
map_single.add_raster(selected_path, cmap="jet", layer_name = selected_file, opacity=1, zoom_to_layer = False, toolbar_control=False, widget_control=False)

In [ ]:
#add the colourmap
map_single.add_colormap("jet", label=f"{selected_file} Chlor_a conc. (mg/m^3)", vmin=min_cmap, vmax = max_cmap, width = 8, height=0.4)

#add the inspector and layer manager
map_single.add_inspector_gui(opened=True, position='bottomleft')

map_single.add_layer_manager(position='topright', opened = True)

In [ ]:
map_single #show single map!